In [2]:
selected_cand = '/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb5_optimizing/output_test_fitnessExtract/gen_1/gen_1_cand_3_data.json'
#split between 'optimizing/ and /gen_
specific_output = selected_cand.split('optimizing/')[1].split('/gen_')[0]

In [5]:
import json
import shutil
import os
from batch_run_files.aw_batch_tools import generate_all_figures
from netpyne import sim
exclude_running = False
output_only = True
fresh_figs = False
file = selected_cand
data_file = file
file = file.replace('_data.json', '_Fitness.json')
assert os.path.exists(file), f'{file} does not exist'
root = os.path.dirname(file)
netP = sim.loadNetParams(data_file, setLoaded=False)

sim.loadAll(data_file)
sim.cfg.duration = 30*1000
filename = sim.cfg.filename
#replace 'output' with 'figure_material'
#filename = filename.replace('output', 'figure_material')
#replace 'output' with 'figure_material'
filename = filename.replace('output', 'figure_material')
#insert specific_output as dir between 'figure_material' and 'gen_'
filename = filename.replace('figure_material', f'figure_material/{specific_output}')
filedir = os.path.dirname(filename)
if not os.path.exists(filedir):
    os.makedirs(filedir)
sim.cfg.filename = filename
#filename = filename.replace('.json', '')

sim.createSimulate(netParams=netP, simConfig=sim.cfg, output=True)
sim.saveData()

Loading file /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb5_optimizing/output_test_fitnessExtract/gen_1/gen_1_cand_3_data.json ... 
  Done; file loading time = 0.16 s
Loading netParams...
Loading file /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb5_optimizing/output_test_fitnessExtract/gen_1/gen_1_cand_3_data.json ... 
  Done; file loading time = 0.12 s
Loading simConfig...
Loading netParams...
Loading net...
  Created 800 cells
  Created 24008 connections
  Created 800 stims
Unable to create NEURON objects: 'E'
  Done; re-instantiate net time = 0.94 s
Loading simData...

Start time:  2024-03-21 16:42:07.293944

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 400 
  Done; cell creation time = 0.06 s.
Making connections...


In [ ]:

for root, dirs, files in os.walk(filename):
    if file.endswith('_datajson'):
        if '.archive' in root:
            continue
        if exclude_running and 'output/' in root: continue
        if output_only and 'output/' not in root: continue
        try:
            fitness = json.load(open(os.path.join(root, file)))
            overall_fitness = fitness['fitness']
        except:
            print(f'Error reading {file}')
        
        if overall_fitness < 400:
            print(f'Overall fitness for {file} is {overall_fitness}')
            print(f'Generating plots for {file}')
            #move_plots_to_plots_folder(root)
            #generate_pdf_reports(root, params)
            #print('Plots and pdf reports generated
        #print(fitness)
            batchdata_path = os.path.join(root, file)
            assert os.path.exists(batchdata_path), f'{batchdata_path} does not exist'
            try:
                simLabel = batchdata_path.split('/')[-1].replace('_data.json', '')
                run_path = os.path.dirname(os.path.dirname(batchdata_path))
                run_basename = os.path.basename(run_path)
                print(f'Generating plots for Candidate: {simLabel}, Run: {run_basename}')

                # fresh_figs = False
                assert os.path.exists(batchdata_path), f'{batchdata_path} does not exist'
                cfg_path = batchdata_path.replace('_data.json', '_cfg.json')
                assert os.path.exists(cfg_path), f'{cfg_path} does not exist'
                # fitness_path = batchdata_path.replace('_data.json', '_Fitness.json')
                # assert os.path.exists(fitness_path), f'{fitness_path} does not exist'

                generate_all_figures(
                    fresh_figs = fresh_figs,
                    net_activity_params = {'binSize': .03*500, 
                                        'gaussianSigma': .12*500, 
                                        'thresholdBurst': 1.0},
                    batchLabel = 'batchRun_evol',
                    #batchLabel = params['filename'][0],
                    #minimum peak distance = 0.5 seconds
                    batch_path = batchdata_path
                )

                output_path = batchdata_path
                output_path = os.path.dirname(output_path)
                output_path = f'{output_path}/NetworkBurst_and_Raster_Figs/'
                run_grand_path = os.path.dirname(run_path)
                plot_row_path = f'{run_grand_path}/goodFit_plots_rows/'
                if not os.path.exists(plot_row_path):
                    os.makedirs(plot_row_path)
                ## shutil copy any files in output_path with the string 'row' in the name to plot path
                for rooti, dirsi, filesi in os.walk(output_path):
                    if '.archive' in rooti: continue
                    
                    for filei in filesi:
                        if 'row' in filei:
                            if os.path.exists(plot_row_path+filei):
                                if fresh_figs: os.remove(plot_row_path+filei)
                                else: continue
                            shutil.copy(rooti+'/'+filei, plot_row_path+filei)
            except:
                print(f'Error generating plots for {file}')
                continue